# 09 Decorators and Closures
Some notes, observations and questions along chapter 09.

"Function decorators let us “mark” functions in the source code to enhance their behavior in some way. This is powerful stuff, but mastering it requires understanding closures—which is what we get when functions capture variables defined outside of their bodies."

### Decorators 101
"A decorator is a callable that takes another function as an argument (the decorated function)."

In [15]:
# both the same

def decorate(func):
    return func

# no. 1
@decorate
def target():
    print('running target()')

target()

# no. 2
def target():
    print('running target()')

target = decorate(target)

target()

running target()
running target()


In [13]:
# a decorated function is in fact replaced:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

target()

running inner()


In [7]:
# inspection reveals that target is a now a reference to inner:
target

<function __main__.deco.<locals>.inner()>

The decorator needs to return a function (or callable) to make `target()` callable, otherwise we get an error. We can return any function, so we can theoretically swap the passed function for something totally different. However, if we want to still use the decorated function, we need to call it from inside `inner()` and return it.

### When Python Executes Decorators

Decorators are executed immediately when a module is loaded, which is usually at import time.

In [21]:
registry = [] # will hold references to functions decorated by @register

def register(func):
    # this is a bit different from usual decorators, in that it is not defining an inner() function
    # and thus cannot change the decorated function; 
    # and also, usually a decorator would be defined in a different file and would be imported
    print(f'running register({func})')
    registry.append(func)
    return func # we must return a function

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main(): # main() is run after the decorated functions are run
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()

running register(<function f1 at 0x703e5eb2c2c0>)
running register(<function f2 at 0x703e5eb2efc0>)
running main()
registry -> [<function f1 at 0x703e5eb2c2c0>, <function f2 at 0x703e5eb2efc0>]
running f1()
running f2()
running f3()


This is the output when this code cell (or we could also imagine it was a file) is run as a script. `register()` runs (twice) before any other function in the module, even before `main()`.

If it is imported, `main()` is not run, but still `register()` is run twice.

    >>> import registration
    --------------
    running register(<function f1 at 0x10063b1e0>)
    running register(<function f2 at 0x10063b268>)

The registry would be filled with f1 and f2.

Main take away: "Decorators are executed as soon as the module is imported, but the decorated functions only run when they are explicitly invoked. This highlights the difference between what Pythonistas call import time and runtime."

#### Question: 
What is the purpose of this? It feels like a bug. (Of cause it isn't.)

#### Answers:
- setup or configuration for the entire module
- control over function behavior before execution: alter function's behavior in advance but without invoking it
- especially useful in situations like registering functions, logging, security checks, or memoization

### How variable scope works in Python
- decorators that want to alter the decorated function's behaviour, do so with an inner function that relies on closures
- to understand closures we are now looking at variable scopes


In [23]:
# surprising example:
b = 6
def f2(a):
    print(a)
    print(b) # the code breaks, because `b` is interpreted as a local variable 
    # (as soon as there is a local variable `b`, the global variable `b` is not taken into account)
    b = 9

f2(3)

3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

"This is not a bug, but a design choice: Python does not require you to declare variables, but assumes that a variable assigned in the body of a function is local."

In [25]:
# to tell interpreter to treat b as a global variable and still assign a new value to it within the function:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

3
6


### Closures
A closure is a function that captures and "remembers" variables from its surrounding (non-global) scope, even after that scope has finished executing. This happens when a function is defined within another function, and the inner function refers to variables from the outer function.

Example: consider a function (in this case a `callable`) that averages over an ever growing list of numbers:

In [27]:
class Averager():

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)
    
avg = Averager()
avg(10)

10.0

In [28]:
avg(11)

10.5

In [29]:
avg(12)

11.0

Instead of a callable, we can also use a higher order function to remember:

In [38]:
def make_averager():
    series = [] # local variable of `make_averager`

    def averager(new_value):
        # we can do this without using `nonlocal` declaration, because `series` is not assigned, only appended to (because it is mutable):
        series.append(new_value) # `averager` can access local variables of its higher order function; in here it is a free variable
        total = sum(series)
        return total / len(series)

    return averager # `make_averager` (factory function) returns the `averager` function

avg = make_averager()
avg(10)

10.0

In [31]:
avg(11)

10.5

In [32]:
avg(12)

11.0

`series` is a free variable in `averager` function, as we can see when inspecing the returned and called `averager` function, which is bound to the variable name `avg`:

In [33]:
#  `__code__` attribute represents the compiled body of the function
avg.__code__.co_varnames

('new_value', 'total')

In [34]:
avg.__code__.co_freevars

('series',)

In [35]:
# value for series is kept in the __closure__ attribute
avg.__closure__

(<cell at 0x703e5f51eaa0: list object at 0x703e5c375080>,)

In [37]:
# each free variable will be found in this list; the first value refers to `series`
# the actual values are found in the `cell_contents` attribute
avg.__closure__[0].cell_contents

[10, 11, 12]

"To summarize: a closure is a function that retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer available."

#### How closures behave in loops

Closures can also be used inside a loop: the closure then captures the last value of the loop variable (it remembers the reference to the variable) and not the current value of the variable during the iteration.

For instance:

In [26]:
def create_closures():
    funcs = []
    for i in range(3):
        def inner():
            return i
        funcs.append(inner)
    return funcs

closures = create_closures()
print([f() for f in closures])  # Output: [2, 2, 2] (not [0, 1, 2])


[2, 2, 2]


We could use the default argument trick to fix this with:
`def inner(i=i):  # Captures the current value of i`

Also see [this example](https://github.com/StefanieSenger/Playground/blob/main/late_binding_clausures.py) from the Hitchiker's Guide to Python, that shows late binding in closures when using a lambda function in a list comprehention.

### The `nonlocal` Declaration
- we use it to assign a value to an immutable type belonging to the higher order function
- lets us declare a variable as a free variable even when it is assigned within the function

This throws an error, because we try to assign to `count` in the body of `averager`:

In [40]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total / count

    return averager

avg = make_averager()
avg(10)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

But this sets count and total as free (`nonlocal`) variables:

In [41]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

avg = make_averager()
avg(10)

10.0

### Implementing a Simple Decorator

In [43]:
import functools
import time

def clock(func):
    @functools.wraps(func) # helper for building well-behaved decorators
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = [repr(arg) for arg in args]
        arg_lst.extend(f'{k}={v!r}' for k, v in kwargs.items())
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked

# this would be another file
@clock
def snooze(seconds):
    time.sleep(seconds)

snooze(3)

[3.00010534s] snooze(3) -> None


"This is the typical behavior of a decorator: it replaces the decorated function with a new function that accepts the same arguments and (usually) returns whatever the decorated function was supposed to return, while also doing some extra processing."

### Decorators in the Standard Library
- `@property`, `@classmethod` and `@staticmethod` are build-in decorators
- `functools` also has `@cache`, `@lru_cache` and `@singledispatch`, which will be treated here

- `@cache` can be applied to painfully slow functions for memoization; `@lru_cache` it its older equivalent from Python <= 3.8, but also has some more functionality like a max_size param which the newer one doesn't have
- `@singledispatch`:
    - "dispatch": determining which implementation of a function to call at runtime, based on certain criteria (usually the type or number of arguments); often associated with method overloading or polymorphism
    - "single dispatch": choosing the function to execute based on the type of one argument (typically the first one)
    - "generic function": function that can work with different types of inputs and may have multiple implementations (or specializations) for different types
    - `@singledispatch` creates a generic function where different implementations of the function can be registered based on the type of the first argument; allows to write type-specific functions
    - used when we want to handle different types of data with different behaviors but keep a single function name

### Parameterized Decorators
- to pass to a decorator other arguments than the function, make a decorator factory that takes those arguments and returns a decorator, which is then applied to the function to be decorated

Simple example without an inner function: registry should only be appended, if `active=True:`

In [44]:
registry = set()

def register(active=True): # register is a decorator factory; it takes an optional kwarg
    def decorate(func): # actual decorator without an inner function
        print('running register'
              f'(active={active})->decorate({func})')
        if active:
            registry.add(func)
        else:
            registry.discard(func)

        return func
    return decorate # our decorator factory returns the decorator `decorate`

@register(active=False) # the `@register` factory must be invoked as a function, with the desired parameters
def f1():
    print('running f1()')

@register() # if no parameters are passed, `register` must still be called as a function
def f2():
    print('running f2()')

def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x703e5cdf93a0>)
running register(active=True)->decorate(<function f2 at 0x703e5cdf84a0>)


In [45]:
# only f2 function appears in the registry
registry

{<function __main__.f2()>}

Without the decorator syntax, using `register` as a regular function, the syntax needed to decorate a function `f` would be `register()(f)` to add `f` to the registry, or `register(active=False)(f)` to not add it.

This is a bit similar to how joblib is used within scikit-learn: `Parallel(n_jobs=self.n_jobs)(delayed(func)())`

Realistic example with an inner function:

In [48]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT): # parametrized factory for decorator function (`fmt` stands for "format")
    def decorate(func): 
        def clocked(*_args): # unpacks args from decorated functions
            t0 = time.perf_counter()
            _result = func(*_args) # unpacks args from decorated functions (`seconds` in the case of `snooze()`)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals())) # using **locals() to allow any local variable of `clocked` to be referenced in the `fmt`
            return _result # clocked will replace the decorated function, so it should return whatever that function returns
        return clocked
    return decorate

if __name__ == '__main__':

    @clock() # will use default argument for `fmt`, since called without argument
    def snooze(seconds):
        time.sleep(seconds)

    for i in range(3):
        snooze(1)

[1.00009542s] snooze(1) -> None
[1.00010351s] snooze(1) -> None
[1.00008300s] snooze(1) -> None


Alternatively, this could be implemented class based using the `__call__` method:

In [55]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

class Clock:

    def __init__(self, fmt=DEFAULT_FMT):
        self.fmt = fmt

    def __call__(self, func): # instance becomes callable; when invoked, the instance replaces the decorated function with `clocked`
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args) # `clocked` wraps the decorated function
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked

clock = Clock()

@clock
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(1)

[1.00009751s] snooze(1) -> None
[1.00009081s] snooze(1) -> None
[1.00009727s] snooze(1) -> None
